In [ ]:
import os
os.environ['KAGGEL_CONFIG_DIR'] ='/content'

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samuelcortinhas/cats-and-dogs-image-classification")

print("Path to dataset files:", path)

100%|██████████| 64.4M/64.4M [00:00<00:00, 80.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/samuelcortinhas/cats-and-dogs-image-classification/versions/4


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  archive.zip
  inflating: test/cats/cat_1.jpg     
  inflating: test/cats/cat_106.jpg   
  inflating: test/cats/cat_109.jpg   
  inflating: test/cats/cat_113.jpg   
  inflating: test/cats/cat_114.jpg   
  inflating: test/cats/cat_116.jpg   
  inflating: test/cats/cat_118.jpg   
  inflating: test/cats/cat_119.jpg   
  inflating: test/cats/cat_124.jpg   
  inflating: test/cats/cat_140.jpg   
  inflating: test/cats/cat_147.jpg   
  inflating: test/cats/cat_156.jpg   
  inflating: test/cats/cat_158.jpg   
  inflating: test/cats/cat_162.jpg   
  inflating: test/cats/cat_18.jpg    
  inflating: test/cats/cat_190.jpg   
  inflating: test/cats/cat_203.jpg   
  inflating: test/cats/cat_223.jpg   
  inflating: test/cats/cat_234.jpg   
  inflating: test/cats/cat_244.jpg   
  inflating: test/cats/cat_251.jpg   
  inflating: test/cats/cat_255.jpg   
  inflating: test/cats/cat_268.jpg   
  inflating: test/cats/cat_279.jpg   
  inflating: test/cats/cat_281.jpg   
  inflating: test/cats/cat_2

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Directory paths
train_dir = '/content/train'
test_dir = '/content/test'

# Preprocess images (resize, convert to grayscale, normalize, flatten)
def preprocess_images(directory, target_size=(64, 64)):
    images = []
    labels = []

    # Loop through subdirectories (cats and dogs)
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)

        # Check if it's a directory
        if os.path.isdir(subdir_path):
            # Loop through images in the subdirectory
            for image_name in os.listdir(subdir_path):
                image_path = os.path.join(subdir_path, image_name)

                # Load and preprocess the image
                img = load_img(image_path, target_size=target_size, color_mode='grayscale')  # Grayscale
                img_array = img_to_array(img) / 255.0  # Normalize image to [0, 1]

                images.append(img_array.flatten())  # Flatten image to vector

                # Assign labels based on subdirectory name
                if subdir == 'cats':
                    labels.append(0)  # 0 for cats
                else:
                    labels.append(1)  # 1 for dogs

    return np.array(images), np.array(labels)

# Preprocess the train and test images
train_images, train_labels = preprocess_images(train_dir)
test_images, test_labels = preprocess_images(test_dir)

# Split the train data into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Normalize the images using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test_images = scaler.transform(test_images)

# Train the SVM model with a linear kernel
svm_model = SVC(kernel='linear', C=1)  # You can experiment with 'rbf' and 'poly' kernels later
svm_model.fit(X_train, y_train)


SVC(C=1, kernel='linear')

In [ ]:
# Evaluate on the validation set
val_predictions = svm_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_val, val_predictions)
class_report = classification_report(y_val, val_predictions)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Validation Accuracy: 62.50%
Confusion Matrix:
[[42 22]
 [20 28]]
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.66      0.67        64
           1       0.56      0.58      0.57        48

    accuracy                           0.62       112
   macro avg       0.62      0.62      0.62       112
weighted avg       0.63      0.62      0.63       112



In [ ]:
# Preprocess test images
test_images, test_labels = preprocess_images(test_dir)

# Flatten test images
test_images_flat = test_images.reshape(test_images.shape[0], -1)
test_images_flat = scaler.transform(test_images_flat)

# Predict the labels for test data
test_predictions = svm_model.predict(test_images_flat)

# Test Accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 50.00%


In [ ]:
svm_model = SVC(kernel='rbf', C=1, gamma=0.5)
svm_model.fit(X_train, y_train)


SVC(C=1, gamma=0.5)